2 Strategies:
- create a dataframe client based to understand the behavior of the client and than try track which features better impact in the target 1 (reactivate)
- create a dataframe with all ocurrences of purchase, to try track which features better impact in the target 1 (reactivate)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

url = '/home/helena/MLFieldWork/DF_ModelH.csv'
target = 'target'
# variables to remove based on distinct values 
variables = ['client_create_date','yyyymm', 'item_id', 'net', 'family_code', 'sales_id', 'sales_net', 'time_diff']

In [3]:
df = pd.read_csv(url)
df.head()

,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,flg_tool,sales_channel,net,family_code,group_code,sales_id,n_purchases,sales_net,time_diff,target
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,-1.0,0
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,XBXV2AB,XBXV2AB0102,9306_201709,2,1579.03,0.0,0
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,0.0,0
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,XBXV1AF,XBXV1AF0201,9306_202010,2,1341.14,1126.0,1
4,8321,2005-11-15,LE,15500,2,681.26,E,T8,2018-04-01,6776,0,B,20.02,XBSI4AA,XBSI4AA0101,8321_201804,7,20.02,-1.0,0


In [5]:
df_pos = df.copy()
df_pos.drop(variables, axis=1, inplace=True)

In [6]:
df_pos.shape

(1933899, 12)

In [7]:
df_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933899 entries, 0 to 1933898
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   client_id      int64  
 1   region         object 
 2   trade_sector   int64  
 3   n_employees    int64  
 4   economic_pot   float64
 5   eco_pot_class  object 
 6   risk_cat       object 
 7   flg_tool       int64  
 8   sales_channel  object 
 9   group_code     object 
 10  n_purchases    int64  
 11  target         int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 177.1+ MB


Use the most frequence value for the columns and the max for the target

In [12]:
def most_frequent(x):
    return x.mode().iloc[0] if not x.mode().empty else pd.NA

# Columns to aggregate by most frequent (excluding target)
columns_to_mode = df_pos.columns.drop(['client_id', 'target'])

# Apply mode aggregation to those columns
agg_mode = df_pos.groupby('client_id')[columns_to_mode].agg(most_frequent)

# For target, take the maximum (1 if they ever reactivated)
agg_target = df_pos.groupby('client_id')['target'].max()

# Combine them back into a single dataframe
client_summary = pd.concat([agg_mode, agg_target], axis=1).reset_index()


In [13]:
client_summary

,client_id,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,flg_tool,sales_channel,group_code,n_purchases,target
0,1,VT,22100,2,4528.46,E,3d,0,D,XAEC4AA0101,1,0
1,2,NO,13500,2,4432.42,E,3d,1,D,XAEN2BA0103,2,0
2,3,BA,21100,3,5000.00,E,3d,1,B,XAEP3CC0301,6,0
3,4,MC,22100,4,5000.00,E,3d,0,A,XBCF1AA0101,1,0
4,5,AN,13500,2,4432.42,E,3d,1,C,XAEC3AA0103,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
85481,93253,MI,36100,1,2538.00,E,5d,0,B,XAEC2CA0601,1,0
85482,93254,ME,11000,7,9772.08,D,5d,1,B,XADC1CA0306,1,0
85483,93255,MI,13500,2,4432.42,E,5d,1,B,XAEP3CC0302,1,0
85484,93256,MB,14400,4,1815.72,E,5d,1,B,XAEP3CA0102,1,0


### Model

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='target')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))